In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install py7zr

In [ ]:
import glob
from PIL import Image
import matplotlib.pyplot as plt
import collections
import math
import os
import shutil
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import datasets, transforms
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

import wandb

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("cifar-10")

wandb.login(key=secret_value_0)
wandb.init(project="Cifar-10", save_code=True)


In [ ]:
!pip install py7zr

In [ ]:
!python -m py7zr x /kaggle/input/cifar-10/train.7z

In [ ]:
!python -m py7zr x /kaggle/input/cifar-10/test.7z

In [ ]:
data_dir = '/kaggle/working/'

In [ ]:
def read_csv_labels(fname):
    """Read `fname` to return a filename to label dictionary."""
    with open(fname, 'r') as f:
        # Skip the file header line (column name)
        lines = f.readlines()[1:]
    tokens = [l.rstrip().split(',') for l in lines]
    return dict(((name, label) for name, label in tokens))

labels = read_csv_labels(os.path.join(data_dir, '/kaggle/input/cifar-10/trainLabels.csv'))
print(f'Number training examples: {len(labels)}')
print(f'Number classes: {len(set(labels.values()))}')

In [ ]:
def copyfile(filename, target_dir):
    """Copy a file into a target directory."""
    os.makedirs(target_dir, exist_ok=True)
    shutil.copy(filename, target_dir)

def reorg_train_valid(data_dir, labels, valid_ratio):
    """Split the validation set out of the original training set."""
    # The number of examples of the class that has the fewest examples in the
    # training dataset
    n = collections.Counter(labels.values()).most_common()[-1][1]
    # The number of examples per class for the validation set
    n_valid_per_label = max(1, math.floor(n * valid_ratio))
    label_count = {}
    for train_file in os.listdir(os.path.join(data_dir, 'train')):
        label = labels[train_file.split('.')[0]]
        fname = os.path.join(data_dir, 'train', train_file)
        copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                     'train_valid', label))
        if label not in label_count or label_count[label] < n_valid_per_label:
            copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                         'valid', label))
            label_count[label] = label_count.get(label, 0) + 1
        else:
            copyfile(fname, os.path.join(data_dir, 'train_valid_test',
                                         'train', label))
    return n_valid_per_label

In [ ]:
def reorg_test(data_dir):
    """Organize the testing set for data loading during prediction."""
    for test_file in os.listdir(os.path.join(data_dir, 'test')):
        copyfile(os.path.join(data_dir, 'test', test_file),
                 os.path.join(data_dir, 'train_valid_test', 'test',
                              'unknown'))

In [ ]:
def reorg_cifar10_data(data_dir, valid_ratio):
    labels = read_csv_labels('/kaggle/input/cifar-10/trainLabels.csv')
    reorg_train_valid(data_dir, labels, valid_ratio)
    reorg_test(data_dir)

In [ ]:
batch_size = 64
valid_ratio = 0.1
reorg_cifar10_data(data_dir, valid_ratio)

In [ ]:
transform_train = torchvision.transforms.Compose([
    # TODO,
    # HorizontalFlip, RandomSizeCrop, "small" Color Jitter
    torchvision.transforms.RandomResizedCrop(32),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3,hue=0.2),
    
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])
])

transform_test = torchvision.transforms.Compose([

    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])])

In [ ]:
train_ds, train_valid_ds = [torchvision.datasets.ImageFolder(
    os.path.join(data_dir, 'train_valid_test', folder),
    transform=transform_train) for folder in ['train', 'train_valid']]

valid_ds, test_ds = [torchvision.datasets.ImageFolder(
    os.path.join(data_dir, 'train_valid_test', folder),
    transform=transform_test) for folder in ['valid', 'test']]

train_iter, train_valid_iter = [torch.utils.data.DataLoader(
    dataset, batch_size, shuffle=True, drop_last=True)
    for dataset in (train_ds, train_valid_ds)]

valid_iter = torch.utils.data.DataLoader(valid_ds, batch_size, shuffle=False,
                                         drop_last=True)

test_iter = torch.utils.data.DataLoader(test_ds, batch_size, shuffle=False,
                                        drop_last=False)

In [ ]:
pretrained_net = torchvision.models.resnet18(pretrained=True)
pretrained_net.fc

In [ ]:
pretrained_net.fc = nn.Linear(pretrained_net.fc.in_features, 10)
nn.init.xavier_normal_(pretrained_net.fc.weight)
nn.init.constant_(pretrained_net.fc.bias, 0);

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
pretrained_net = pretrained_net.to(device)
if device == 'cuda':
    pretrained_net = torch.nn.DataParallel(pretrained_net) # if multiple GPUs use them
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(pretrained_net.parameters(), lr=1e-4, weight_decay=0.0001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, verbose=True, min_lr=1e-5, factor=0.5)

In [ ]:
from tqdm.notebook import trange, tqdm


epochs = 10
for epoch in trange(epochs):
    accurate = 0
    total = 0
    losses = 0
    for X, y in tqdm(train_iter):
        X = X.to(device)
        y = y.to(device)
        y_pred = pretrained_net(X)
        loss = criterion(y_pred, y)
        score, predicted = torch.max(y_pred, 1)
        accurate += (y == predicted).sum().float()
        losses += loss.item()
        total += len(y)

        # zero the gradients before running
        # the backward pass.
        optimizer.zero_grad()

        # Backward pass to compute the gradient
        # of loss w.r.t our learnable params. 
        loss.backward()

        # Update params
        optimizer.step()
    
    wandb.log({
            'train_loss': losses / len(train_iter),
            'train_accuracy': accurate / total
    })
    
    
    with torch.no_grad():
        pretrained_net.eval()
        accurate = 0
        total = 0
        losses = 0
        for X, y in tqdm(valid_iter):
            X = X.to(device)
            y = y.to(device)
            y_pred = pretrained_net(X)
            loss = criterion(y_pred, y)
            score, predicted = torch.max(y_pred, 1)
            accurate += (y == predicted).sum().float()
            losses += loss.item()
            total += len(y)
            
        
        wandb.log({
            'valid_loss': losses / len(valid_iter),
            'valid_accuracy': accurate / total,
        })

In [ ]:
predictions = []
pretrained_net.eval()
with torch.no_grad():
    for X, _ in tqdm(test_iter):
        X = X.to(device)
        predictions.extend(pretrained_net(X).argmax(dim=1).type(torch.int32).cpu().numpy())
ids = list(range(1, len(test_ds)+1))
ids.sort(key=lambda x: str(x))

df = pd.DataFrame({'id': ids, 'label': predictions})
df['label'] = df['label'].apply(lambda x: train_ds.classes[x]) 
df.to_csv('submission.csv', index=False)

In [ ]:
torch.save(pretrained_net.state_dict(), 'pretrained_net.pt')